<a href="https://colab.research.google.com/github/Shakilkhan24/Playground_DL/blob/main/langchain_agent_with_tools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain
!pip install langchain-openai
!pip install langchain-experimental

In [ ]:
!pip install langchainhub

In [ ]:
!pip install wikipedia

In [ ]:
!pip install -U duckduckgo-search

In [29]:
from langchain.tools import StructuredTool

In [ ]:
import os
from langchain.prompts import PromptTemplate
from langchain import hub
from langchain.agents import Tool, AgentExecutor, create_react_agent,StructuredTool
from langchain.tools import DuckDuckGoSearchRun, WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper
from langchain_experimental.tools.python.tool import PythonAstREPLTool
from langchain_openai import ChatOpenAI
api_key='sk-proj-JU7iMY44eT7jq1tYpLYWT3BlbkFJxNQCnciCwFcjVSuTlPYp'
llm = ChatOpenAI(temperature=0,openai_api_key=api_key)

template = '''
Answer the following question as best you can.
Questions: {question}
'''
prompt_template = PromptTemplate.from_template(template)

prompt = hub.pull('hwchase17/react')

# Create the tools
python_repl = PythonAstREPLTool()
python_repl_tool = Tool(
    name='Python REPL',
    func=python_repl.run,
    description='Use when you need to run Python to answer a question. You Should input Python code.'
)

api_wrapper = WikipediaAPIWrapper()
wikipedia = WikipediaQueryRun(api_wrapper=api_wrapper)
wikipedia_tool = Tool(
    name='Wikipedia',
    func=wikipedia.run,
    description='Use for when you need to look up a topic, country, or person on Wikipedia.'
)

search = DuckDuckGoSearchRun()
duckduckgo_tool = Tool(
    name='DuckDuckGo Search',
    func=search.run,
    description='Use for when you need to perform an internet search to find information that another tool can not provide.'
)

tools = [python_repl_tool, wikipedia_tool, duckduckgo_tool]

agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=3
)

# Define the question
question = 'What are the next 10 numbers in the Fibonacci sequence starting from 5'

output = agent_executor.invoke({'input': prompt_template.format(question=question)})

print(output)

In [ ]:
print(prompt)

In [ ]:
question='What are the biggest rivers?'
output = agent_executor.invoke({'input': prompt_template.format(question=question)})

print(output)

In [ ]:
import torch
import torch.nn.functional as F
question='transformer large language model, write self attention code'
output = agent_executor.invoke({'input': prompt_template.format(question=question)})

print(output)

In [38]:
from langchain.pydantic_v1 import BaseModel, Field
class FINE(BaseModel):
  s:str=Field(description='give a string input')

def func(s:str):
  return len(s)

tool = StructuredTool.from_function(
    func=func,
    name="length finder",
    description="returning length of string",
    args_schema=FINE,
    return_direct=True,
    # coroutine= ... <- you can specify an async method if desired as well
)

In [44]:
tools = [tool]

agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=1
)

In [45]:
question='What are the biggest rivers?'
output = agent_executor.invoke({'input': prompt_template.format(question=question)})

print(output)



> Entering new AgentExecutor chain...
I need to find the length of the input question to determine the answer.
Action: length finder
Action Input: "What are the biggest rivers?"28


> Finished chain.
{'input': '\nAnswer the following question as best you can.\nQuestions: What are the biggest rivers?\n', 'output': 28}


In [43]:
len(question)

28

In [65]:
import numpy as np
import pandas as pd
from langchain.pydantic_v1 import BaseModel,Field

# we could have given a csv url instead
# here we just experimented with pydantic and the tool method , experiment purpose actually


class Input(BaseModel):
  df: str = Field(description='give csv file path')

def df_des_finder(df_path: str) -> pd.DataFrame:
  df = pd.read_csv(df_path)
  return df.describe()

tool = StructuredTool.from_function(
    func=df_des_finder,
    name='description finder',
    description='We are going to find the description about the dataframe given ',
    args_schema=Input,
    return_direct=True
)

tools = [tool]

agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=2
)

question='the csv file path : /content/size.csv '
output = agent_executor.invoke({'input': prompt_template.format(question=question)})

print(output)



> Entering new AgentExecutor chain...
I need to find the description of the dataframe in the size.csv file.
Action: description finder
Action Input: df_path = "/content/size.csv"

FileNotFoundError: [Errno 2] No such file or directory: 'df_path = "/content/size.csv'

{'df': {'title': 'Df', 'description': 'give csv file path', 'type': 'string'}}



```
# Giving any url and crawling all the links from there , AGENT
```



In [71]:
import numpy as np
import pandas as pd
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import StructuredTool
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import requests

class Input(BaseModel):
    url: str = Field(description='give url')

def crawl_links(url: str) -> list:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    base_url = url
    links = [urljoin(base_url, a['href']) for a in soup.find_all('a', href=True) if a['href']]
    return links

tool = StructuredTool.from_function(
    func=crawl_links,
    name='Link Crawler',
    description='Crawl all links from a given URL',
    args_schema=Input,
    return_direct=True
)

tools = [tool]

agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=2
)

question='the url is : https://python.langchain.com/docs/modules/agents/quick_start/ '
output = agent_executor.invoke({'input': prompt_template.format(question=question)})
print(output)



> Entering new AgentExecutor chain...
I should use the Link Crawler tool to gather information from the provided URL.
Action: Link Crawler
Action Input: "https://python.langchain.com/docs/modules/agents/quick_start/"['https://python.langchain.com/docs/modules/agents/quick_start/#__docusaurus_skipToContent_fallback', 'https://python.langchain.com/', 'https://python.langchain.com/docs/modules/', 'https://python.langchain.com/docs/integrations/platforms/', 'https://python.langchain.com/docs/guides/', 'https://api.python.langchain.com', 'https://python.langchain.com/docs/modules/agents/quick_start/', 'https://python.langchain.com/docs/people/', 'https://python.langchain.com/docs/packages/', 'https://python.langchain.com/docs/contributing/', 'https://python.langchain.com/docs/templates/', 'https://github.com/langchain-ai/langchain/blob/master/cookbook/README.md', 'https://python.langchain.com/docs/additional_resources/tutorials/', 'https://python.langchain.com/docs/additional_resources/yo

In [ ]:
!pip install --upgrade --quiet  arxiv

In [74]:
from langchain.agents import AgentExecutor, create_react_agent, load_tools
tools = load_tools(
    ["arxiv"],
)
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [75]:
question='different attention mechanism in transformer models'
output = agent_executor.invoke({'input': prompt_template.format(question=question)})
print(output)



> Entering new AgentExecutor chain...
I should search for scientific articles on arxiv.org that discuss different attention mechanisms in transformer models.
Action: arxiv
Action Input: different attention mechanism in transformer modelsPublished: 2022-09-30
Title: Adaptive Sparse and Monotonic Attention for Transformer-based Automatic Speech Recognition
Authors: Chendong Zhao, Jianzong Wang, Wen qi Wei, Xiaoyang Qu, Haoqian Wang, Jing Xiao
Summary: The Transformer architecture model, based on self-attention and multi-head
attention, has achieved remarkable success in offline end-to-end Automatic
Speech Recognition (ASR). However, self-attention and multi-head attention
cannot be easily applied for streaming or online ASR. For self-attention in
Transformer ASR, the softmax normalization function-based attention mechanism
makes it impossible to highlight important speech information. For multi-head
attention in Transformer ASR, it is not easy to model monotonic alignments in
different